## Step 4: Python code to post online

We set up some code to post data to the online spreadsheet.

The data comes from the Comms channel in the form of a dictionary:
``` 
comm_data = {'time': date-time value, 
             'temp': value, 
             'humidity': value,
             'ratio': value,
             'lux': value}
```
This will get sent to the function **post_stuff()**, which decides when to post it based on how much time has elapsed. We record data more frequently in the daytime, since the sunlight makes interesting things happen. 

We also have a widget to keep track of whether data has been posted. 

In [39]:
# the URL of the spreadsheet, for posting
post_url = 'https://ethercalc.net/_/MyWeirdName23'

# set a reminder of when the last data was stored
last_time = datetime.now() - timedelta(hours=2)

# check if it is daytime
def is_daylight(): 
    if (datetime.now().hour > 8) and (datetime.now().hour < 20):
        return True
    return False

# a widget to monitor the data being posted
monitor = widgets.Text(
    value="No recent post yet",
    placeholder="blank",
    description="Latest data:",
    disabled = False
    )

# here we grab the data results from comm, check if it is time to post, then post. 
def post_stuff(result):
    global last_time
    if is_daylight():
        wait_time = 14*60 # in daytime, we post every 15 minute (wait 14). 
    else:
        wait_time = 59*60 # in nighttime, post every 60 minutes (wait 59)
    if (datetime.now()-last_time).seconds < wait_time:
        return
    last_time = datetime.now()
    data = datetime.now().strftime("%Y-%m-%d,%H:%M:%S")
    data += ',' + str(result['temp'])
    data += ',' + str(result['humidity'])
    data += ',' + str(result['ratio'])
    data += ',' + str(result['lux'])
    monitor.value = data    
    r = requests.post(post_url, data= data) ## we could return a status code here...

## Step 5: Open a comms channel 

This connects the Javascript and Python engines.

Javascript (front end) and Python (back end) run separately on the Jupyter system. We open a Comms channel between the two so JS can pass the sensor data to Python, then Python posts it onto the web.  

There is an issue that Python runs as a single thread in Jupyter. So if you put a "sleep" command in a Python loop, no other code can access the Jupyter kernel during that loop. The sensor data is then not recorded. 

The solution is to create a timer loop in Javascript, so that every 5 minutes it sends a message to the Python kernel. Basically, the message says "hey Python, here is the latest data. Please save it to the cloud." Then the Python code can run and save the data. 

This way there is no loop or sleep in Python, so nothing to interupt. The loop is in JS (See "setIntevalTimer in the JS code below).


### First open the front end of the Comms channel, in JavaScript

In [20]:
%%js
Jupyter.notebook.kernel.comm_manager.register_target('my_comm_target',
    function(comm, msg) {
        // comm is the frontend comm instance, msg is the comm_open message, which can carry data
        // We don't do anything with the above opening msg
        // Next, we register a handler to deal with later messages.
        // Basically, on receiving a message, we start a timer that posts a send command,
        // which sends the data in a message to the Python back end
        comm.on_msg(function(msg) {
            console.log("Comm message received " + msg.content.data.foo);
            // this function is called by the timer, sends data to the back end
            function myTimer() {      
                const d = new Date();
                comm.send({'time': d.toLocaleString(), 
                           'temp':tempSensor.temperature, 
                           'humidity':humSensor.humidity,
                           'ratio':vrSensor.sensorValue,
                           'lux':liteSensor.illuminance});
            } 
            let myVar = setInterval(myTimer, 300000); // run every 5 minutes (300sec)
        });
        comm.on_close(function(msg) {return 0;});
    });

<IPython.core.display.Javascript object>

### Next, open the back end of the Comms channel, in Python

In [21]:
from ipykernel.comm import Comm

# a global variable we use to keep the data from the front end comm channel
comm_data = "Not set yet"  

# Connect to the comm channel in the front end, to the Python
my_comm = Comm(target_name='my_comm_target', data={'foo': 1})

# Add a callback for received messages. This will call post_stuff to store the data
@my_comm.on_msg
def _recv(msg):
    global comm_data
    comm_data = msg['content']['data']  # this is the data in the comm message
    post_stuff(comm_data)